In [1]:
# activate package, basic imports + load data
include("/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/behavior/analysis_fixtures.jl")
include("/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/behavior/analysis_visuals.jl")
using Dierckx

import jcontrol: State, movingaverage

  Activating project at `~/Documents/Github/LocomotionControl/analysis/behavior`


@Info (Main):  Loaded 1021 trials 


  ╰──────────────────────────────────────────────── 
                      Tue, 10 May 2022 09:40:28 
@Info (Main):  After tortuosity analysis, discarded 9.109% of trials | 928

 
  │                trials left 
  ╰──────────────────────────────────────────────── 
                      Tue, 10 May 2022 09:40:31 


## Generate dataset for RNN training
It takes the tracking data processed through the Julia behavior analysis code and creates a dataset for training an RNN on.

The frist step is to get the trial's trajectory in the track's curvilinear coordinates system and to upsample the data so that they are
expressed at an arbitrary framerate $\Delta t$.

Next, we sample `N` random time points from each trial and take the next `T` seconds of the trial as a training sample (to avoid training over an entire trial).
Then, at each frame in the selected chunk of trial we get the track's curvature at each $\Delta s$ point from the mouse's current position to `S` cm later.

In [2]:
# PARAMS
Δt = 0.005  # time step duration of upsampled trial
N = 3    # number of chunks per trial
T = .66  # duration of each trial "chunk" for training
Δs = 2.5  # distance between sampling's of the track's curvature
S = 20   # lookahead distance for track's curvature.

20

### Prepare tracking data
Load tracking data, turn into track's curvilinear coordinates and upsample

In [3]:

function upsample_framerate(x, Δt)
    t_original = 0:1/60:(length(x)/60 + 1)
    t_original = t_original[1:length(x)]
    t_upsample = 1:Δt:(t_original[end])
    spl = Spline1D(t_original, x; k=2) # k: order of interpolant; can be between 1-5
    return spl(t_upsample)
end

mutable struct CurvilinearCoordsTrials
    x::Vector
    y::Vector
    s::Vector
    n::Vector
    ψ::Vector
    V::Vector  # velocity vector: V = √(v² + u²)
    ω::Vector
    V̇::Vector  # acceleration
    ω̇::Vector  # angular acceleration
end

function CurvilinearCoordsTrials(states::Vector{State})
    v = upsample_framerate(getfield.(states, :v).^2 + getfield.(states, :u).^2, Δt)
    ω = upsample_framerate(getfield.(states, :ω), Δt)

    v̇ = movingaverage(diff(v), 21) / Δt
    ω̇ = movingaverage(diff(ω), 21) / Δt

    return CurvilinearCoordsTrials(
        upsample_framerate(getfield.(states, :x), Δt),
        upsample_framerate(getfield.(states, :y), Δt),    
        upsample_framerate(getfield.(states, :s), Δt),
        upsample_framerate(getfield.(states, :n), Δt),
        upsample_framerate(getfield.(states, :ψ) .- getfield(states[1], :ψ), Δt),
        upsample_framerate(v, Δt),
        upsample_framerate(ω, Δt),
        v̇,
        ω̇,
    )

end

CurvilinearCoordsTrials

In [4]:
trial_states(trial) = map(frame->State(trial, frame, FULLTRACK; v=trial.v[frame]), 1:length(trial.x))
ctrials = map(trial -> CurvilinearCoordsTrials(trial_states(trial)), trials)
nothing

### Step 2
Cut the trials into chunks.

In doing so, make sure not to take chunks that are too close of the trial to avoid having problems.

In [5]:
chunks = []
for trial in ctrials
    # timesteps = 1:findlast(trial.s .< (trial.s[end]-(S+15)))
    # timesteps = timesteps[1:end - (Int ∘ round)(T * 1000)]
    timesteps = 1:(length(trial.s) - (Int ∘ round)(T * 1000))
    if length(timesteps) == 0
        continue
    end

    starts = (Int ∘ round).(rand(timesteps, N))
    stops = (Int ∘ round).(starts .+ (T * 1000))


    for (start, stop) in zip(starts, stops)
        if stop <= start || stop >= length(trial.n)
            continue
        end
        push!(chunks, 
            CurvilinearCoordsTrials(
                trial.x[start:stop],
                trial.y[start:stop],
                trial.s[start:stop],
                trial.n[start:stop],
                trial.ψ[start:stop],
                trial.V[start:stop],
                trial.ω[start:stop],
                trial.V̇[start:stop],
                trial.ω̇[start:stop],
            )
        )
    end
end

@info "Got $(length(chunks)) chunks out of $(length(trials)) trials"
(length(trials) * 3 - length(chunks))/3

@Info (Main):  Got 2752 chunks out of 928 trials 
  ╰──────────────────────────────────────────────── 
                      Tue, 10 May 2022 09:45:19 


10.666666666666666

### Step 3
for each chunk get the track's curvature ahead at each frame

In [6]:
struct DatasetEntry
    n::Vector{Float64}
    ψ::Vector{Float64}
    s::Vector{Float64}
    V::Vector{Float64}
    ω::Vector{Float64}
    V̇::Vector{Float64}
    ω̇::Vector{Float64}
    k::Matrix{Float64}   # stores the curvature at each waypoint and each frame
end

In [10]:
waypoints = collect(0:Δs:S)
waypoints_idxs = 1:length(waypoints)

entries = []
for chunk in chunks
    # get curvature
    chunk_curv = zeros(length(waypoints_idxs), length(chunk.s))

    for frame in 1:length(chunk.x)
        # get relevant S vector from track
        s̄ = FULLTRACK.S .- chunk.s[frame]
        start = argmin(s̄)
        stop = findlast(s̄ .<= S)
        @assert stop > start
        @assert FULLTRACK.S[stop] <= (chunk.s[frame]+S)

        svec = FULLTRACK.S[start:stop]

        for (I, sval) in zip(waypoints_idxs, waypoints)
            idx = findfirst((svec .- svec[1]) .> sval)
            # push!(chunk_curv[I], svec[idx])
            chunk_curv[I, frame] = svec[idx]
        end
    end

    push!(entries, DatasetEntry(
        chunk.n,
        chunk.ψ,
        chunk.s,
        chunk.V,
        chunk.ω,
        chunk.V̇,
        chunk.ω̇,
        chunk_curv,
    )
    )

end

In [8]:
@info "Got $(length(entries)) entries from $(length(chunks)) chunks"

@Info (Main):  Got 0 entries from 2752 chunks 
  ╰──────────────────────────────────────────────── 
                      Tue, 10 May 2022 09:45:25 


### Save data
save data to file in a format which can be loaded back in python for RNN training

In [9]:
using JSON: JSON
using DataFrames: DataFrame
import JSONTables: objecttable, jsontable

savepath = "/Users/federicoclaudi/Dropbox (UCL)/Rotation_vte/Locomotion/analysis/RNN/dataset"

for (i, entry) in enumerate(entries)
    ks = collect(Symbol("k_", i)=>entry.k[i, :] for i in 1:size(entry.k,1 ))
    dict = Dict{Symbol, Union{Vector{Float64}, Matrix{Float64}}}(:n=>entry.n, :ψ=>entry.ψ, :s=>entry.s, :V=>entry.V, :ω=>entry.ω, ks...)
    df = DataFrame(dict)
    open(joinpath(savepath, "$(i).json"), "w") do f
        write(f, objecttable(df))
    end
end